In [6]:
import pandas as pd

df = pd.read_csv('./combined_flight_data.csv')

print(df.head())




  Departure Airport       Departure Time Arrival Airport         Arrival Time  \
0               SGN  2024-09-15 05:25:00             HAN  2024-09-15 07:35:00   
1               SGN  2024-09-15 06:00:00             HAN  2024-09-15 08:10:00   
2               SGN  2024-09-15 06:40:00             HAN  2024-09-15 08:50:00   
3               SGN  2024-09-15 07:00:00             HAN  2024-09-15 09:10:00   
4               SGN  2024-09-15 08:10:00             HAN  2024-09-15 10:20:00   

  Carrier Code  Flight Number  Price (VND) Duration  
0           VJ            198    1541000.0  PT2H10M  
1           VJ            120    1541000.0  PT2H10M  
2           VJ            168    1541000.0  PT2H10M  
3           VJ            122    1541000.0  PT2H10M  
4           VJ            124    1541000.0  PT2H10M  


In [17]:
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

mydb = mysql.connector.connect(
  host=os.getenv('DB_HOST'),
  user=os.getenv('DB_USER'),
  password=os.getenv('DB_PASSWORD'),
  database=os.getenv('DB_NAME'),
)

mycursor = mydb.cursor()

print("Đã kết nối thành công đến MySQL!")



InterfaceError: 2003: Can't connect to MySQL server on '%-.100s:%u' (%s) (Warning: %u format: a real number is required, not str)

In [ ]:
print(f"DB_HOST: {os.getenv('DB_HOST')}")
print(f"DB_USER: {os.getenv('DB_USER')}")
print(f"DB_PASSWORD: {os.getenv('DB_PASSWORD')}")
print(f"DB_NAME: {os.getenv('DB_NAME')}")

DB_HOST: 127.0.0.1:3306
DB_USER: dinhgia2106
DB_PASSWORD: Dinhgia2106.
DB_NAME: flight-db


In [18]:
mydb = mysql.connector.connect(
  host=('localhost'),
  user=('root'),
  password=('Dinhgia2106.'),
)

print(mydb.cursor())

ProgrammingError: 1045 (28000): Access denied for user ''@'localhost' (using password: NO)